In [1]:
import pandas as pd 
import sqlalchemy as sql
import alpaca_trade_api as tradeapi 
import os
from dotenv import load_dotenv
#import hvplot.pandas 
from pandas_datareader import data  #remember to pip install pandas-datareader
import pandas_datareader as pdr 
import math 

%matplotlib inline 

In [2]:
symbol='USO','TSLA', 'SPY'
start='2010-06-29'
end='2021-10-15'

df=data.get_data_yahoo(symbol, start, end)['Close']
df.head()

Symbols,USO,TSLA,SPY
Date,,,
2010-06-29,273.359985,4.778,104.209999
2010-06-30,271.679993,4.766,103.220001
2010-07-01,262.959991,4.392,102.760002
2010-07-02,260.799988,3.840,102.199997
2010-07-06,260.880005,3.222,102.870003


In [4]:
df['USO_pct_change']= df['USO'].pct_change()
df['TSLA_pct_change']= df['TSLA'].pct_change()
df['SPY_pct_change']= df['SPY'].pct_change()


df.describe()


Symbols,USO,TSLA,SPY,USO_pct_change,TSLA_pct_change,SPY_pct_change
count,2846.000000,2846.000000,2846.000000,2845.000000,2845.000000,2845.000000
mean,164.771809,106.112510,226.313131,-0.000287,0.002437,0.000568
std,102.387174,189.260934,82.173761,0.022700,0.035274,0.010644
min,17.040001,3.160000,102.199997,-0.253150,-0.210628,-0.109424
25%,85.120003,10.464000,158.595005,-0.010686,-0.014816,-0.003469
50%,112.760002,45.909000,209.770004,0.000690,0.001255,0.000649
75%,276.460007,65.295502,279.069992,0.011073,0.019081,0.005452
max,361.200012,883.090027,453.190002,0.166667,0.243951,0.090603


In [6]:
# Working with SQL in order to not ping the Alpaca API too much  
#Creating SQL connection/engine

database_connection_string = 'sqlite:///'
engine = sql.create_engine(database_connection_string)
engine

Engine(sqlite:///)

In [7]:
#shoving closing_prices_df from Alpaca into a SQL Table 

df.to_sql('closing_prices', engine, index= True, if_exists='replace' )


In [8]:
#Code to grab the data from the SQL table into a pandas df to analyze further 

closing_prices_df = pd.read_sql_table ('closing_prices', con=engine)


In [13]:
closing_prices_df.describe()

,USO,TSLA,SPY,USO_pct_change,TSLA_pct_change,SPY_pct_change
count,2846.000000,2846.000000,2846.000000,2845.000000,2845.000000,2845.000000
mean,164.771809,106.112510,226.313131,-0.000287,0.002437,0.000568
std,102.387174,189.260934,82.173761,0.022700,0.035274,0.010644
min,17.040001,3.160000,102.199997,-0.253150,-0.210628,-0.109424
25%,85.120003,10.464000,158.595005,-0.010686,-0.014816,-0.003469
50%,112.760002,45.909000,209.770004,0.000690,0.001255,0.000649
75%,276.460007,65.295502,279.069992,0.011073,0.019081,0.005452
max,361.200012,883.090027,453.190002,0.166667,0.243951,0.090603
